In [1]:
import pandas as pd
import numpy as np

In [21]:
from sklearn.base import BaseEstimator
from sklearn.utils.extmath import squared_norm

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [5]:
df_mod = pd.read_csv('dados/df_60_mod_cr.csv')
df_mod.head().style.applymap(lambda x: 'white-space:nowrap')

,loan_amnt,int_rate,installment,emp_length,home_ownership,verification_status,dti,total_acc,earl_cr_line_t,loan_inc_pct,log_income,tempo_mes,status,status_repayment
0,16625,17.270000,415.600000,5.000000,MORTGAGE,Verified,18.090000,22.000000,-296 days +18:24:00,0.415729,10.596385,38.000000,1,0
1,35000,19.720000,921.850000,10.000000,MORTGAGE,Verified,12.040000,55.000000,-344 days +01:36:00,0.364583,11.472103,9.000000,0,1
2,32000,17.770000,808.600000,10.000000,MORTGAGE,Verified,9.500000,34.000000,-206 days +00:48:00,0.320000,11.512925,14.000000,0,1
3,16100,15.310000,385.650000,6.000000,RENT,Source Verified,10.800000,18.000000,-151 days +20:00:00,0.335417,10.778956,29.000000,0,1
4,20000,23.630000,571.080000,8.000000,RENT,Verified,24.790000,26.000000,-322 days +08:00:00,0.357143,10.933107,16.000000,0,1


Testando com uma amostra dos dados

In [6]:
def get_train_data(df_mod, n_sample):
    
    df_mod = df_mod.sample(n=n_sample, random_state=1234)

    X = df_mod.loc[:,['loan_amnt', 'int_rate', 'installment']]
    y = df_mod.loc[:, ['tempo_mes', 'status', 'status_repayment']]

    return X, y

In [7]:
X_mod, y_mod = get_train_data(df_mod, n_sample=20)

Preparando o que vamos precisar para o loop no gradient

In [8]:
def prepare_y(y_mod):

    y1 = y_mod[['status', 'status_repayment', 'tempo_mes']].to_numpy()
    # array: list to tuple
    aux = [(e1, e2, e3) for e1, e2, e3 in y1]

    new_y = np.array(aux, dtype=[('evento_0', '?'), ('evento_1', '?'), ('tempo', '<f8')])

    return new_y

In [9]:
new_y = prepare_y(y_mod)

In [11]:
# cria variaveis necessarias para o loop no boosting e gradient
# (para visualizacao)
def vec_for_gradient(y_mod):

    n_sample = y_mod.shape[0]

    gradient_0 = np.zeros(n_sample)
    gradient_1 = np.zeros(n_sample)

    exp_tsj = np.zeros((n_sample))
    y_pred = np.zeros(n_sample)
    exp_pred = np.exp(y_pred)

    return gradient_0, gradient_1, exp_tsj, y_pred, exp_pred

gradient_0, gradient_1, exp_tsj, y_pred, exp_pred = vec_for_gradient(y_mod)

Riscos "na mão".
Posteriormente, encapsular os riscos competitivos em um só loop

In [19]:
# cada risco deve ter um exp_pred
def negative_gradient_cox_cr(y, y_pred, sample_weight=None):
    """
    Negative gradient of partial likelihood
    """

    n_samples = y.shape[0]

    gradient_0 = np.zeros(n_samples)
    gradient_1 = np.zeros(n_samples)

    exp_tsj_0 = np.zeros((n_samples))
    exp_tsj_1 = np.zeros((n_samples))
    
    # cada risco deve ter um exp_pred!!!!
    exp_pred_0 = np.exp(y_pred['gradient_0'])
    exp_pred_1 = np.exp(y_pred['gradient_1'])

    y_tempo = y['tempo']
    y_evento_0 = y['evento_0']
    y_evento_1 = y['evento_1']

    # --------------------------------------------------------------------------
    # risco 0 (default)
    for i in range(n_samples):
        for j in range(n_samples):
            if y_tempo[j] >= y_tempo[i]:
                exp_tsj_0[i] += exp_pred_0[j]

    for i in range(n_samples):
        s0 = 0
        for j in range(n_samples):
            if (y_evento_0[j] and not y_evento_1[j]) and y_tempo[i] >= y_tempo[j]:
                #s += exp_pred[i] / exp_tsj[j]
                s0 = s0 + exp_pred_0[i] / exp_tsj_0[j]
        gradient_0[i] = y_evento_0[i] - s0
    # --------------------------------------------------------------------------

    # --------------------------------------------------------------------------
    # risco 1 (repayment)
    for i in range(n_samples):
        for j in range(n_samples):
            if y_tempo[j] >= y_tempo[i]:
                exp_tsj_1[i] += exp_pred_1[j]
    
    for i in range(n_samples):
        s1 = 0
        for j in range(n_samples):
            if (y_evento_1[j] and not y_evento_0[j]) and y_tempo[i] >= y_tempo[j]:
                #s += exp_pred[i] / exp_tsj[j]
                s1 = s1 + exp_pred_1[i] / exp_tsj_1[j]
        gradient_1[i] = y_evento_1[i] - s1
    # --------------------------------------------------------------------------

    if sample_weight is not None:
        gradient_0 *= sample_weight
        gradient_1 *= sample_weight

    gradient_cr = np.fromiter(
        zip(gradient_0, gradient_1), dtype=[('gradient_0', np.float64), ('gradient_1', np.float64)]
        )

    return gradient_cr

Agora, o passo é ajustar o _fit_stage_componentwise

In [15]:
class ComponentwiseLeastSquares(BaseEstimator):

    def __init__(self, component):
        self.component = component

    def fit(self, X, y, sample_weight):
        
        xw = X[:, self.component] * sample_weight
        b = np.dot(xw, y)
        if b == 0:
            self.coef_ = 0
        else:
            a = np.dot(xw, xw)
            self.coef_ = b / a

        return self

    def predict(self, X):
        return X[:, self.component] * self.coef_

In [16]:
def fit_stage_cwl(X, residuals, sample_weight):
    
    n_features = X.shape[1]
    base_learners = []
    error = np.empty(n_features)


    for component in range(n_features):
        learner = ComponentwiseLeastSquares(component).fit(X, residuals, sample_weight)
        l_pred = learner.predict(X)
        error[component] = squared_norm(residuals - l_pred)
        base_learners.append(learner)


    best_component = np.nanargmin(error)
    best_learner = base_learners[best_component]
    return best_learner

Definindo X, residuo (pseudo-y) e sample weight

In [17]:
n_samples = new_y.shape[0]
Xi = np.column_stack((np.ones(n_samples),  X_mod))
sw = np.ones(n_samples, dtype=np.float32)
# residuals sendo o gradiente do evento 0 (default)

Rodando o modelo para riscos competitivos

In [22]:
n_estimators = 1000
learning_rate = 0.1
y_pred = np.fromiter(
    zip(np.zeros(new_y.shape[0]), np.zeros(new_y.shape[0])),
    dtype=[('gradient_0', np.float64), ('gradient_1', np.float64)]
)

best_learner_0 = []
best_learner_1 = []

n_risk = 2
estimadores = [[] for _ in range(n_risk)]

for i in range(int(n_estimators)):

    # Revisare e confirmar se esta tudo ok
    # 
    # devemos definir os residuals aqui, pois ele deve ser atualizado!!!!
    # vamos precisar de um y_pred para cada risco!!!!!
    # --------------------------------------------------------------------------
    
    # Passo 1: y = tempo, y_pred = 0
    #   - obtem: vec_grad_1
    #
    # Passo 2: y = tempo, y_pred = residuo da regressao (vec_grad_1 ~ x_i)
    #   - obtem: vec_grad_2
    #
    # Passo 3: y = tempo, y_pred = residuo da regressao (vec_grad_2 ~ x_i)
    #   - obtem: vec_grad_3
    # ...
    # ...
    # ...
    #
    residuals = negative_gradient_cox_cr(y=new_y, y_pred=y_pred)

    # Regressao usando o que sai do gradient como y
    best_learner_0 = fit_stage_cwl(Xi, residuals['gradient_0'], sw)
    best_learner_1 = fit_stage_cwl(Xi, residuals['gradient_1'], sw)

    estimadores[0].append(best_learner_0)
    estimadores[1].append(best_learner_1)

    y_pred['gradient_0'] = y_pred['gradient_0'] + learning_rate * best_learner_0.predict(Xi)
    y_pred['gradient_1'] = y_pred['gradient_1'] + learning_rate * best_learner_1.predict(Xi)

In [23]:
y_pred['gradient_0']
np.exp(y_pred['gradient_0'])

array([2.98449749, 1.19787585, 1.40411674, 2.55163144, 1.92091803,
       2.30462201, 2.36457576, 1.61755208, 2.32860351, 2.90176132,
       2.79144643, 1.57198723, 2.0708024 , 1.89453838, 1.78782072,
       2.23867289, 2.4581036 , 2.69432136, 2.36097853, 2.18831174])

array([19.77656187,  3.31307199,  4.07192858, 12.82801491,  6.82722321,
       10.02038992, 10.6395241 ,  5.04073586, 10.26359848, 18.20618411,
       16.30458618,  4.8162096 ,  7.93118453,  6.64947819,  5.97641396,
        9.38087355, 11.68263557, 14.79547455, 10.60132009,  8.92014089])

Após o ajuste do modelo, precisamos de:
- IBS: predict_survival_function(x_test)
- C-index: score(x_test, y_test)
- AUC: predict(x_test)